In [87]:
with open('shakespeare.txt','r') as f:
    text = f.read()

In [167]:
class BytePairEncoder():
    def __init__(self,vocab_size):
        self.vocab_size = vocab_size
        self.merges = {}
        self.vocab = {}
    
    def _build_vocab(self):
        self.vocab = {i:bytes([i]) for i in range(256)}
        for (x,y),idx in self.merges.items():
            self.vocab[idx] = self.vocab[x] + self.vocab[y]
    
    def get_vocab(self):
        vocab = {k:v.decode('utf-8',errors='replace') for k,v in self.vocab.items()}
        return vocab
    
    def ids_to_token(self,ids):
        tokens = []
        for idx in ids:
            tokens.append(self.vocab[idx].decode('utf-8',errors='replace'))
        return tokens
    def get_pair_counts(self,tokens):
        counts = {}
        for x,y in zip(tokens,tokens[1:]):
            counts[(x,y)] = counts.get((x,y),0) + 1
        return counts
    def fit(self,text):
        self.merges = {}
        self.vocab = {}
        tokens = list(map(int,text.encode('utf-8')))
        ids = list(tokens)
        for i in range(self.vocab_size - 255):
            tokens = self.get_pair_counts(ids)
            pair = max(tokens,key=tokens.get)
            idx = 256 + i
            self.merges[pair] = idx
            enc = []
            i=0
            while i <len(ids):
                if i < len(ids)-1 and (ids[i] == pair[0] and ids[i+1] == pair[1]):
                    enc.append(idx)
                    i+=2
                else:
                    enc.append(ids[i])
                    i+=1
            idx+=1
            ids = enc
    def encode(self, x):
        x_bytes = x.encode('utf-8')
        ids = list(x_bytes)

        while len(ids) >= 2:
            pair_counts = self.get_pair_counts(ids)
            pair = min(pair_counts, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break
            enc = []
            idx = self.merges[pair]
            i = 0
            while i < len(ids):
                if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
                    enc.append(idx)
                    i += 2
                else:
                    enc.append(ids[i])
                    i += 1
            ids = enc
        return ids

    
    def decode(self,ids):
        if not self.vocab:
            self._build_vocab()
        tokens = b"".join([self.vocab[i] for i in ids])
        return tokens.decode('utf-8',errors='replace')

In [201]:
enc = BytePairEncoder(vocab_size=1000)

In [202]:
enc.fit(text)

In [203]:
ids = enc.encode(text[:1000])

In [204]:
print(enc.decode(ids))

THE SONNETS

by William Shakespeare

From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And tender churl mak'st waste in niggarding:
Pity the world, or else this glutton be,
To eat the world's due, by the grave and thee.

When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a tattered weed of small worth held:  
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say within thine own deep sunken eyes,
Were an all-eating shame, and thriftless prais

In [205]:
print('-'.join(enc.ids_to_token(ids)))

T-H-E- -S-O-N-N-E-T-S-

-by -W-ill-i-am- -S-ha-k-es-p-ear-e-

-From- fa-ir-est -c-rea-tur-es -w-e -des-ir-e -inc-reas-e,
That -there-by -beauty's -r-ose -m-ight -ne-ver- d-i-e,
But -as -the -ri-per- should -by -time -dec-eas-e,
-H-is -tend-er- h-e-ir- m-ight -bear- his -m-e-mor-y-:
-But -thou -cont-rac-ted -to- th-ine own- b-right -eyes-,
-F-eed-'st -thy -light-'s -f-la-me -with- s-elf---sub-st-an-ti-al- f-u-el-,
-M-ak-ing a- f-am-ine -where -ab-und-ance -li-es-,
Th-y -self- thy -fo-e, -to- thy -sweet -self -too -c-ru-el-:
-Thou- that -art -now- the -wor-ld-'s -f-res-h- -orn-am-ent-,
And -on-ly -her-a-ld -to the -g-a-ud-y -s-pr-ing-,
W-ith-in th-ine own- b-u-d -b-uri-est -thy -cont-ent-,
And -tend-er -ch-ur-l- m-ak-'st -w-ast-e -in -n-ig-g-ar-d-ing-:
-P-it-y -the -wor-ld-, -or -el-s-e th-is -gl-ut-t-on- b-e,
-To -ea-t -the -wor-ld-'s -d-u-e, -by -the -gra-ve -and -th-ee-.

-Wh-en- for-ty -w-inter-s -sha-ll- b-es-i-e-g-e thy -b-row-,
And -di-g- d-eep- -tr-en-ch-es -in thy -beauty's -fi-